In [1]:
import sys
import time
import random
import numpy as np
from functools import reduce
from itertools import permutations

# RANDOM SEARCH
### 1) Given a target sequence of characters (e.g. 'hello world')
### 2) Create a set of characters, *S* (e.g. ['h', 'e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd'])
### 3) Until target is found...
### 4) Produce a guess by randomly combining characters from *S* to form a sequence

In [2]:
def random_search(trgt):
    
    target, letters = list(trgt), list(trgt)
    guess, count = [], 0
    print('TARGET: {}'.format(trgt))
    start = time.time()
    
    while guess != target:
        guess = list(np.random.choice(letters, size=len(target), replace=False))
        if count%500 == 0:
            print('\rGUESS : {}'.format(reduce(lambda x,y: x+y, guess)), end='')
        sys.stdout.flush()
        count += 1
        
    end = time.time()
    print('\rOUTPUT: {}'.format(reduce(lambda x,y: x+y, guess)), end='')
    print('\nCOUNT: {}\nTIME: {}'.format(count+1, end-start))
    
    return end-start

# PERMUTATION SEARCH

### 1) Generate a list (L) of permutations of the target
### 2) Shuffle L
### 3) Until target is found...
### 4) Produce a guess by randomly choosing from L
### 5) Remove incorrect guess from L

In [3]:
def permutation_search(trgt):
    
    target = tuple(trgt)
    perms = list(permutations(target)) # get a list of permutations
    random.shuffle(perms)
    guess, count = random.choice(perms), 0
    print('TARGET: {}'.format(trgt))
    start = time.time()
    
    while guess != target:
        perms.remove(guess)
        guess = random.choice(perms)
        print('\rGUESS : {}'.format(reduce(lambda x,y: x+y, guess)), end='')
        sys.stdout.flush()
        count += 1
        
    end = time.time()
    print('\rOUTPUT: {}'.format(reduce(lambda x,y: x+y, guess)), end='')
    print('\nCOUNT: {}\nTIME: {}'.format(count+1, end-start))
    
    return end-start

# GENETIC ALGORITHM
### 1) Randomly generate initial population
### 2) Until target is found in population...
### 3) Determine fitness of population
### 4) Reproduce to generate new population
### 5) Mutate new population
### 6) Replace old population with new population 

In [4]:
def fitness(pop, target):
    '''Calculate fitness of each member of population'''
    # dictionary in which to store fitness scores
    results = {}
    
    # for each member of the population
    for member in pop:
        score = 0
        # determine the number of character matches (character order matters)
        for index, val in enumerate(member):
            if val == target[index]:
                score += 1
        results[tuple(member)] = score/len(member)
    
    return results


def crossOver(N, chars, prob_list, mutation_rate):
    '''Mate parents to produce child and apply mutation.'''
    new_population = []
    for _ in range(N):
        
        # randomly choose two parents
        ind_1, ind_2 = np.random.choice(len(prob_list), size=2, replace=False)
        parent_1, parent_2 = prob_list[ind_1], prob_list[ind_2]
        
        # take parts from each parent
        midpoint = np.random.randint(1, len(parent_1))
        child = list(parent_1)[:midpoint]
        child.extend(list(parent_2)[midpoint:])
        
        # mutate child
        for i in range(len(child)):
            if np.random.random(size=1) <= mutation_rate:
                child[i] = chr(random.choice(chars))
        new_population.append(child)
    
    return new_population


def genetic_algorithm(target, N=200, mutation_rate=0.01):
    
    target = list(target)
    
    # characters to choose from
    characters = list(range(97,122))
    characters.append(32)
    
    # randomnly generate population
    population = []
    for _ in range(N):
        population.append(list(map(chr, list(np.random.choice(characters,
                                                              size=len(target))))))
    
    num_generations = 1
    print('TARGET: {}'.format(reduce(lambda x,y: x+y, target)))
    start = time.time()
    
    while target not in population:
        
        # get fitness scores for population
        results = fitness(population, target)
        
        # determine most fit population member
        leader = max(results, key=lambda x: results[x])
        print('\rOUTPUT: {}\t\tFITNESS: {}\t\tNUM GENERATIONS: {}'.format(
              reduce(lambda x,y: x+y, leader), round(results[leader],3), num_generations), end='')
        
        # create probability list according to fitness
        prob_list = []
        for result in results:
            # append member to prob_list as many times as determined by its fitness score
            for _ in range(int(results[result]*100)):
                prob_list.append(result)
        
        # reproduce
        population = crossOver(N, characters, prob_list, mutation_rate)
        num_generations += 1
        
    end = time.time()
    print('\rOUTPUT: {}\t\tFITNESS: {}\t\tNUM GENERATIONS: {}'.format(
          reduce(lambda x,y: x+y, target), '1.000', num_generations), end='')
    print('\nTime: {}'.format(end-start))
    return end-start

In [5]:
msg = 'hi world'
msg = msg.lower()
print('RANDOM SEARCH\n')
rs_time = random_search(msg)
print('-'*50)
print('\nPERMUTATION SEARCH\n')
ps_time = permutation_search(msg)
print('-'*50)
print('\nGENETIC ALGORITHM\n')
ga_time = genetic_algorithm(msg)
print('\n\nGenetic Algorithm performed approximately {} times faster than Random Search.'.\
      format(int(rs_time/ga_time)))
print('\nGenetic Algorithm performed approximately {} times faster than Permutation Search.'.\
      format(int(ps_time/ga_time)))

RANDOM SEARCH

TARGET: hi world
OUTPUT: hi world
COUNT: 53265
TIME: 10.431010007858276
--------------------------------------------------

PERMUTATION SEARCH

TARGET: hi world
OUTPUT: hi world
COUNT: 15161
TIME: 22.102240085601807
--------------------------------------------------

GENETIC ALGORITHM

TARGET: hi world
OUTPUT: hi world		FITNESS: 1.000		NUM GENERATIONS: 11
Time: 0.4301598072052002


Genetic Algorithm performed approximately 24 times faster than Random Search.

Genetic Algorithm performed approximately 51 times faster than Permutation Search.
